In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import scipy.optimize as optimize
from scipy.optimize import LinearConstraint
from random import random
from collections import Counter
from random import random
import scipy.optimize as optimize
from scipy.stats import betabinom
import pickle
from IPython.display import display, HTML
from colorsys import rgb_to_hls, hls_to_rgb
from collections import defaultdict


In [ ]:

def read_conformations_file(n_aminos, dimension):
    return pd.read_csv(os.path.join("conformations", dimension, str(n_aminos)+".csv"))


def proper_count_hist(ax, data, return_highest_prob=False, **kwargs):
    rvs_counter = Counter(data)
    rvs_realized, rvs_realized_count = zip(*list(rvs_counter.items()))
    rvs_realized_prob = [elem/sum(rvs_realized_count) for elem in rvs_realized_count]

    if return_highest_prob:
        return ax.bar(rvs_realized, rvs_realized_prob, **kwargs), max(rvs_realized_prob)
    else:
        return ax.bar(rvs_realized, rvs_realized_prob, **kwargs)



def plot_distribution(ax, n_aminos, dimension, return_highest_prob=False, **kwargs):
    conformation_df = read_conformations_file(n_aminos, dimension)

    return proper_count_hist(ax, conformation_df["num_collisions"], return_highest_prob=return_highest_prob, **kwargs)


def lighten_color(color):
    """Lighten a color by half the difference between the current color and white."""
    # Convert color to RGB
    r, g, b = [x / 255.0 for x in tuple(int(color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4))]

    # Convert RGB to HSL
    h, l, s = rgb_to_hls(r, g, b)

    # Calculate the difference between current color and white
    l_diff = 1.0 - l

    # Lighten the color by half of the difference
    new_l = min(1, l + l_diff / 2)

    # Convert the new HSL back to RGB
    new_r, new_g, new_b = hls_to_rgb(h, new_l, s)

    # Convert RGB back to hexadecimal color code
    return "#{:02x}{:02x}{:02x}".format(int(new_r * 255), int(new_g * 255), int(new_b * 255))


In [ ]:
from bb_fit import get_fit, cmap_from_colorblind

# colors = [['#4d9221', '#b8e186'],
# ['#2b8cbe', '#9ebcda'],
# ['#d73027', '#fdae61']]


# def new_cmap(i, j):
#     return colors[i][j]


# settings for reproducing figure 3 of "Can HP-protein Folding Be Solved with Genetic Algorithms? Maybe not"
colors = [['#4d9221', '#b8e186'],
['#2b8cbe', '#9ebcda'],
['#d73027', '#fdae61']]
alpha_values = [0.7, 0.4] # (for first color, for second color)

FACECOLOR = "#B5B5B5" # legend boxes & box with n=*length*
BOX_ALPHA = 0.7 # legend boxes & box with n=*length*
BOX_STYLE = "round,pad=0.2,rounding_size=0.05" # legend boxes & box with n=*length*
EDGE_COLOR = "black" # legend boxes & box with n=*length*
fontweight = 500 # text for sigma
fontweight_bold = 600 # text for sigma
linestyle_mu_vline = ":" # vertical lines
linestyle_distribution_fit = "--" # fitted betabinomial line)
legend_padding = .5 # legend boxes & box with n=*length*

def subplot_increase_ylim(ax, fraction_increase=.1):
    # Get current y-limit
    ymin, ymax = ax.get_ylim()

    # Calculate 10% of the range
    range_10_percent = fraction_increase * (ymax - ymin)

    # Increase the upper limit by 10% of the range
    new_ymax = ymax + range_10_percent
    print("ymin, ymax, new_ymax",ymin, ymax, new_ymax)

    # Set the new y-limit
    ax.set_ylim(ymin, new_ymax)


# fontsize = 14.5
fontsize = 14.5
plt.rcParams.update({'font.size': fontsize})
distr_param_size = 12
# left_legend_fontsize = fontsize
# left_legend_fontsize = 11
left_legend_fontsize = distr_param_size
dimension_text_size = left_legend_fontsize

# plotted_dims = "2D 3D 5D 9D".split()


# for legend_placement in ["corner", "bottom"]:
#     for num_plotted_dims in [3, 4]:
#         for plotted_lengths in [[100,150,200], [100,150]]:
for legend_placement in ["corner"]:
    for num_plotted_dims in [3]:
        for plotted_lengths in [[100,150,200]]:
            num_plotted_lengths = len(plotted_lengths)
            max_num_collisions = 0
            max_prob = 0
            plotted_dims = ("2D 3D 4D 5D".split())[:num_plotted_dims]

            alpha_values = [.7, .4, .3]
            fontweight = 600

            figure_height = 3*(len(plotted_dims) + 0.5*int(legend_placement == "bottom"))


            fig, ax = plt.subplots(nrows=len(plotted_dims), ncols=1, figsize=(9/.91/.93,figure_height))
            plwise_textboxes = defaultdict(list)
            if legend_placement == "bottom":
                handles = []
            for enum_dimension, dimension in enumerate(plotted_dims):

                cur_ax = ax[enum_dimension]
                if legend_placement == "corner":
                    handles = []
                max_max_prob = 0

                for enum_aminos, n_aminos in enumerate(plotted_lengths):


                    conformation_df = read_conformations_file(n_aminos, dimension)
                    dim_pl_conflicts = conformation_df["num_collisions"]

                    max_num_collisions = conformation_df["num_collisions"].max()

                    cur_ax.set_xlim(0,max_num_collisions)

                    fit_results = get_fit(n_aminos, dimension, kind="reparameterize")
                    a, b, mu_fit, mu_data, std_fit, std_data = fit_results["a"], fit_results["b"], fit_results["mu_fit"], fit_results["mu_data"], fit_results["std_fit"], fit_results["std_data"],

                    cur_ax.axvline(mu_fit, linestyle=linestyle_mu_vline, color='black', dashes=(1,5))

                    bbf_handle = cur_ax.plot(np.arange(max_num_collisions+1), betabinom.pmf(np.arange(max_num_collisions+1), n_aminos, a,b), color='black', linestyle=linestyle_distribution_fit, label="beta binomial fit")[0]

                    ccc = cmap_from_colorblind(int(dimension[:-1]), enum_aminos)
                    if legend_placement == "corner":
                        distribution_label = f"N = {n_aminos}"
                    elif legend_placement == "bottom":
                        distribution_label = f"{dimension}, N = {n_aminos}"
                    else:
                        raise ValueError(f"expected legend_placement in ['corner', 'bottom'], got {legend_placement}")
                    handle, max_prob = plot_distribution(cur_ax, n_aminos, dimension, label=distribution_label, color=ccc, alpha=alpha_values[enum_aminos], return_highest_prob=True, linewidth=0.8, edgecolor=ccc, width=1)
                    max_max_prob = max(max_max_prob, max_prob)

                    handles.append(handle)

                    mu_text = r'$\mathbf{\mu}$'+f" = {round(mu_fit, 2)}, ({round(mu_data, 2)})"
                    mu_len = len(mu_text)
                    movement_factor = 0.2/3
                    text_x_mu = mu_fit + max_num_collisions*0.06 + max_num_collisions*(enum_aminos * movement_factor)
                    text_y_mu = (1 - (enum_aminos + .10)/(len(plotted_dims)+.6))*cur_ax.get_ylim()[1]
                    if dimension == "2D":
                        if enum_aminos == 0:
                            if distr_param_size == 12:
                                text_y_mu -= 0.0024
                            elif distr_param_size == 11:
                                text_y_mu -= 0.0018
                            text_x_mu += max_num_collisions/40
                        if enum_aminos == 1:
                            if num_plotted_lengths == 2:
                                text_y_mu = .057
                            else:
                                text_y_mu = .073
                                text_x_mu += -max_num_collisions/40
                        if enum_aminos == 2:
                            text_x_mu = 55


                    text_x_sigma = text_x_mu + max_num_collisions*0.02
                    text_y_sigma = text_y_mu-max_max_prob*0.1
                    t1 = cur_ax.text(text_x_mu, text_y_mu, mu_text, size=distr_param_size, ha='left', va='top', fontweight=fontweight, )
                    t2 = cur_ax.text(text_x_sigma, text_y_sigma, r'$\sigma$'+f" = {round(std_fit, 2)}, ({round(std_data, 2)})", size=distr_param_size, ha='left', va='top', fontweight=fontweight-100)

                    # text_x_mu = mu_fit+mu_len*.36
                    # text_y_mu = max_prob
                    # text_x_sigma = mu_fit+mu_len*.36
                    # text_y_sigma = max_prob-max_prob*0.084
                    # t1 = cur_ax.text(text_x_mu, text_y_mu, mu_text, ha='center', va='top', fontweight=fontweight)
                    # t2 = cur_ax.text(text_x_sigma, text_y_sigma, r'$\sigma$'+f" = {round(std_fit, 2)}, ({round(std_data, 2)})", ha='center', va='top', fontweight=fontweight-100)

                subplot_increase_ylim(cur_ax)
                # lgnd = cur_ax.legend(handles=handles, loc="upper right", ncols=len(plotted_dims)+1)
                plot_size = np.array([plt.gca().bbox.width, plt.gca().bbox.height])

                # legend_size = lgnd.get_bbox_to_anchor().transformed(plt.gca().transAxes).bounds[2:] / plot_size  # Size of the legend box in pixels

                # Calculate legend coordinates
                distance = 10  # Equidistant distance from top and right
                legend_x = 1 - distance / plot_size[0]
                legend_y = 1 - distance / plot_size[1]


                # Place the legend
                if legend_placement == "corner":
                    legend = cur_ax.legend(handles=handles + [bbf_handle], loc='upper right', bbox_to_anchor=(legend_x, legend_y), facecolor=FACECOLOR, ncols=1, edgecolor=EDGE_COLOR, borderpad=legend_padding,  prop={'size': left_legend_fontsize})

                # bbox_style = legend.get_frame().get_boxstyle()
                # bbox_style.pad = legend_padding

                plt.draw()
                subplot_header = cur_ax.text(0.5, .95, f"{dimension}", transform=cur_ax.transAxes, ha='center', va='top', fontweight=fontweight, bbox=dict(facecolor=FACECOLOR, alpha=BOX_ALPHA, edgecolor=EDGE_COLOR, pad=legend_padding*6), size=dimension_text_size)
                plt.draw()

                # f = legend.get_frame()
                # w0, h0 = f.get_width(), f.get_height()
                # inv = fig.transFigure.inverted()
                # w, h = inv.transform((w0, h0))
                # n_equals = cur_ax.text(0.5, 1-h, f"{dimension}", transform=cur_ax.transAxes, ha='center', va='top', fontweight=fontweight, bbox=dict(facecolor=FACECOLOR, edgecolor='black', alpha=BOX_ALPHA, boxstyle=bbox_style))
                # n_equals = cur_ax.text(0.5, legend_y, f"n={n_aminos}", transform=cur_ax.transAxes, ha='center', va='top', fontweight=fontweight, bbox=dict(facecolor=FACECOLOR, edgecolor='black', alpha=BOX_ALPHA, boxstyle=bbox_style))

            empty_handle = cur_ax.plot([],[], color="none", label=" ")[0]
            empty_handle.label = " "
            # if legend_placement == "bottom":
            #     handles.append(bbf_handle)
            #     handles.append(empty_handle)
            #     lgnd = plt.legend(handles=handles, bbox_to_anchor=(.5, -.2), loc="upper center", ncols=len(plotted_dims)+1, facecolor=FACECOLOR, borderpad=legend_padding)
            #     lgnd.get_frame().set_alpha(BOX_ALPHA)
            #     lgnd.get_frame().set_edgecolor(EDGE_COLOR)
            # plt.subplots_adjust(bottom=0.3)

            plt.draw()


            xmax = 0
            for enum_dimension, Dimension in enumerate(plotted_dims):
                ax[enum_dimension].tick_params(axis='y', direction='in', which='both')
                # ax[enum_dimension].xaxis.set_ticklabels([])
                xmax = max(xmax, max(ax[enum_dimension].get_xlim()))

            for enum_dimension, Dimension in enumerate(plotted_dims):
                ax[enum_dimension].set_xlim(0, xmax)
                if enum_dimension != (len(plotted_dims) - 1):
                    ax[enum_dimension].set_xticks([])


            ax[-1].set_xlabel("Constraint violations ('collisions')")


            plt.tight_layout()
            # plt.savefig(os.path.join("fits", f"vertical d={','.join([str(d) for d in plotted_dims])} l={','.join([str(l) for l in plotted_lengths])} legend={legend_placement}"), dpi=600)
            plt.savefig(os.path.join("fits", f"vertical distributions d={','.join([str(d) for d in plotted_dims])} l={','.join([str(l) for l in plotted_lengths])} legend={legend_placement}"), dpi=600)
            plt.show()

In [ ]:
from matplotlib.gridspec import GridSpec


def create_axes_required_samples(subplot="2x2+1"):

    # Generate some data
    x = np.linspace(0, 10, 100)
    y1 = np.sin(x)
    y2 = np.cos(x)

    if subplot == "2x2+1":
        # idea paper
        # Create figure and GridSpec
        fig = plt.figure(figsize=(8/5*6/.91, 6))
        gs = GridSpec(2, 6, figure=fig)

        # First subplot spanning two rows
        ax_0_0 = fig.add_subplot(gs[0, :2])
        ax_0_1 = fig.add_subplot(gs[0, 2:4])
        ax_1_0 = fig.add_subplot(gs[1, :2])
        ax_1_1 = fig.add_subplot(gs[1, 2:4])
        ax_01_2 = fig.add_subplot(gs[:, 4:])

        return [ax_0_0, ax_0_1, ax_1_0, ax_1_1, ax_01_2]
    elif subplot == "old":
        # Create figure and GridSpec
        fig = plt.figure(figsize=(10, 3))
        gs = GridSpec(1, 3, figure=fig)

        # First subplot spanning two rows
        ax1 = fig.add_subplot(gs[0, 0])
        ax2 = fig.add_subplot(gs[0, 1])
        ax3 = fig.add_subplot(gs[0, 2])

        return [ax1, ax2, ax3]


axs = create_axes_required_samples(subplot="old")
plt.tight_layout()
plt.show()


In [ ]:
required_samples = pd.read_csv("required_samples.csv")

fontsize = 14.5
# left_legend_fontsize = fontsize
left_legend_fontsize = 11
right_legend_fontsize = left_legend_fontsize
plt.rcParams.update({'font.size': fontsize})

l = []
logspace = np.logspace(0,-11,200)
for i in range(20):
    l.append(logspace[(i+1)*10-1])
l

xticks_positions = [0,50,100,150,200]
xticks_labels = list(map(str, xticks_positions))

min_0_std_range = True
# collision_text_placement = "inside"
collision_text_placement = "outside"

old = False
use_geom_average = True
fit_kinds = ["individual"]
p0_dict = {}

for fit_kind in ["reparameterize"]:
    if old:
        plotted_dims = [str(d)+"D" for d in [2,3]]
        axs = create_axes_required_samples(subplot="old")
    else:
        axs = create_axes_required_samples()
        plotted_dims = [str(d)+"D" for d in [2,3,4,5]]

    for enum_dimension, dimension in enumerate([int(N_d[:1]) for N_d in plotted_dims]):
        dimension_str = str(dimension)+"D"
        ccc = cmap_from_colorblind(dimension, 0)
        p0s = []


        means, stds, mins, maxs = [], [], [], []
        lengths = []
        for length in sorted(required_samples["length"].unique().tolist()):
            lengths.append(length)
            fit_results = get_fit(length, dimension_str, fit_kind, overwrite=False, verbose=False)
            # fit_results = get_fit(length, dimension_str)
            a, b, mu_fit, mu_data, std_fit, std_data = fit_results["a"], fit_results["b"], fit_results["mu_fit"], fit_results["mu_data"], fit_results["std_fit"], fit_results["std_data"],
            _min, _max = fit_results["min"], fit_results["max"]
            means.append(mu_fit)
            stds.append(std_fit)
            mins.append(_min)
            maxs.append(_max)
            p0s.append(betabinom.pmf(0,length,a,b))

        means = np.array(means)
        stds = np.array(stds)

        axs[enum_dimension].plot(lengths, means, color=ccc, label=r"$\mu$")
        if min_0_std_range:
            axs[enum_dimension].fill_between(lengths, means+stds, [max(0,e) for e in (means-stds).tolist()], label=r"$\mu\pm\sigma$", alpha=0.4, color=ccc, edgecolor='black')
        else:
            axs[enum_dimension].fill_between(lengths, means+stds, means-stds, label=r"$\mu\pm\sigma$", alpha=0.4, color=ccc, edgecolor='black')
        axs[enum_dimension].fill_between(lengths, mins, maxs, label="min-max", alpha=0.2, color=ccc, edgecolor='black')
        ymin, ymax = axs[enum_dimension].get_ylim()
        x_min, x_max = axs[enum_dimension].get_xlim()
        if collision_text_placement == "inside":
            x_pos = x_min + 0.20*(x_max - x_min)
            axs[enum_dimension].text(x_pos, (ymin + ymax) / 2, 'collisions', rotation=90, verticalalignment='center')
            axs[enum_dimension].tick_params(axis='x', direction='in', which='both')
            axs[enum_dimension].tick_params(axis='y', direction='inout', which='both', pad=-25, right=False, left=True)
            axs[enum_dimension].set_xlabel(f"{dimension_str} Protein Length")
            axs[enum_dimension].set_xticks(xticks_positions, xticks_labels)
        elif collision_text_placement == "outside":
            x_pos = x_min + 0.04*(x_max - x_min)
            # axs[enum_dimension].text(x_pos, (ymin + ymax) / 2, 'collisions', rotation=90, verticalalignment='center')
            # axs[enum_dimension].text(x_pos, (ymin + ymax) / 2, 'collisions', rotation=90, verticalalignment='center')
            axs[enum_dimension].tick_params(axis='x', direction='in', which='both')
            axs[enum_dimension].tick_params(axis='y', direction='inout', which='both', pad=-24, right=False, left=True)
            axs[enum_dimension].set_ylabel('collisions')
            # axs[enum_dimension].tick_params(axis='y', direction='inout', which='both', right=False, left=True)
            axs[enum_dimension].set_xlabel(f"{dimension_str} Protein Length")
            axs[enum_dimension].set_xticks(xticks_positions, xticks_labels)

        lgnd = axs[enum_dimension].legend(loc="upper center", borderpad=legend_padding,  prop={'size': left_legend_fontsize})
        # lgnd = axs[enum_dimension].legend(loc="upper center", borderpad=legend_padding,  prop={'size': left_legend_fontsize})
        lgnd.get_frame().set_alpha(BOX_ALPHA)
        lgnd.get_frame().set_edgecolor(EDGE_COLOR)
        lgnd.get_frame().set_facecolor(FACECOLOR)
        lgnd.pad = legend_padding


        ddf = required_samples[required_samples["dimension"] == dimension]

        required = []
        lengths = []
        for (length, lddf) in ddf.groupby("length"):
            if lddf["samples_required"].isna().any():
                continue
            required.append(1 / lddf["samples_required"].median())
            lengths.append(length)
        print(required)

        axs[-1].scatter(lengths, required, c=ccc)
        # axs[-1].plot(lengths, required, label=dimension_str, c=ccc)
        all_lengths = [(i+1)*10 for i in range(20)]
        axs[-1].plot(all_lengths, p0s, label=dimension_str, c=ccc)
        p0_dict[dimension_str] = {"p0s":p0s, "all_lengths": all_lengths}



    # axs[-1].plot([(i+1)*10 for i in range(20)],l, color="red", label="fitline")
    axs[-1].tick_params(axis='x', direction='in', which='both')
    axs[-1].set_xticks(xticks_positions, xticks_labels)
    axs[-1].scatter([],[],label="data", color="black")
    axs[-1].axhline(1/1_000_000, linestyle="--", color="black", label="limit")
    lgnd = axs[-1].legend(loc="lower left", borderpad=legend_padding,  prop={'size': right_legend_fontsize})
    lgnd.get_frame().set_alpha(BOX_ALPHA)
    lgnd.get_frame().set_edgecolor(EDGE_COLOR)
    lgnd.get_frame().set_facecolor(FACECOLOR)
    lgnd.pad = legend_padding
    lgnd.pad = 200
    plt.draw()


    plt.yscale("log")
    plt.tight_layout()
    plt.savefig(os.path.join("fits", f"2x2+1_collision_distr_required_samples_fitkind={fit_kind}.png"), dpi=600)


In [ ]:
# parametrizations

- Estimates for probability of 0 violations are saved to "./p_violations=0.txt"
- Estimates for "mean_violations_params.txt"

In [ ]:

def exp_func(lengths, exponent, scalar):
    return scalar * np.exp(-exponent * lengths)


def format_exp_func(popt):
    # see: exp_func_scalar
    return f"P(violations=0|n) = {popt[1]} exp(-{popt[0]} n)"

from scipy.optimize import curve_fit


descriptors = []
for d, dim_dict in p0_dict.items():
    # dim_dict = {"p0s":p0s, "all_lengths": all_lengths}
    lens, p0s =  np.array(dim_dict["all_lengths"]), np.array(dim_dict["p0s"])
    # plt.plot(lens, p0s, label=d + "ground truth")

    popt_with_scalar, pcov = curve_fit(exp_func, lens, p0s)
    print(pcov)
    # plt.plot(lens, exp_func_scalar(lens, *popt_with_scalar), label=d + "exponential with scalar (prev paper)")
    descriptors.append(str((d, format_exp_func(popt_with_scalar))))
    # plt.legend()
    # plt.yscale("log")
    # plt.show()

with open("p_violations=0.txt", "w+") as f:
    f.write("\n".join(descriptors))


def format_mu_reparameterize(x0,x1,x2,x3):
    return r"\mu = " f"exactly: ({x0}n+{x1})/({x0}n+{x1} + {x2}n+{x3})n", f"roughly:  {x0/(x0+x2)}n"

from collections import defaultdict
pd = [str(d)+"D" for d in [2,3,4,5,6,7,8,9]]
mu_formatted = []
for fit_kind in ["reparameterize", "direct"]:
    for enum_dimension, dimension in enumerate([int(N_d[:1]) for N_d in pd]):
        dimension_str = str(dimension)+"D"
        ccc = cmap_from_colorblind(dimension, 0)
        for length in sorted(required_samples["length"].unique().tolist()):
            fit_results = get_fit(length, dimension_str, "reparameterize", overwrite=False, verbose=False)

            a, b, mu_fit, mu_data, std_fit, std_data = fit_results["a"], fit_results["b"], fit_results["mu_fit"], fit_results["mu_data"], fit_results["std_fit"], fit_results["std_data"],
            x0,x1,x2,x3 = fit_results["x0"], fit_results["x1"], fit_results["x2"], fit_results["x3"]
            format_mu_reparameterize(x0,x1,x2,x3)
            mu_formatted.append((str(dimension)+"D", fit_kind, format_mu_reparameterize(x0,x1,x2,x3)))

            break

write_mean_violations_est_str= "\n".join(str(s) for s in mu_formatted)
with open("mean_violations_params.txt", "w+") as f:
    f.write(write_mean_violations_est_str)
